In [ ]:
"""
https://docs.google.com/spreadsheets/d/1rPVuSU3EizUDdjFlT6NUvJAhl0x102GfUELDRrH1tQA/edit#gid=0

TODO:
    No entiende de contextos ¿semánticos?, p.e: "Fui a Madrid de Austria" no pone art
    BERT no pone artículos indefinidos
    Ver las reglas ya puestas mirando el libro
    Tratar Lo
    Ver si funciona bien los casos al/del
    repasar corpus
"""

In [1]:
############### PREPARAR CORPUS ###############

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.nn.init as init

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
############### CARGAR MODELOS ###############


import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.eval()


INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/raconma/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/raconma/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/raconma/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpg2uiz_ur
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_prob

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
  

In [3]:
############### CARGAR FRASES ###############

lines = open('/home/raconma/TFM/corpus/corpus_parlamento_sin_articulos.txt', encoding='utf8')
lines = lines.read().splitlines()

In [44]:
############### PREPARAR FRASES ###############

"""
https://nlp.stanford.edu/software/spanish-faq.html

Adjectives: a
Conjunctions: c
Determiners: d
Punctuation: f
Nouns: n
Pronouns: p
Adverbs: r
Prepositions: s
Verbs: v
Dates: w
Numerals: z
"""
def verbo_anterior(anteriores):
    anteriores.reverse()
    verbo = ''
    for word in anteriores:
        if word[1].startswith('v'):
            verbo = word

articulos = ["el", "la", "los", "las", "un", "una", "unos", "unas", "lo"]
articulo_lo = "lo"
pronombres_relativos = ["que", "cual", "cuales"]

def sentence_masker(frase_taggeada, print_word):
    anteriores = [("","")]
    frase_con_mask = ['[CLS]']
    
    for word in frase_taggeada:
            
        # SI ANTES DE ADV+ADJ NO HAY NI SUST NI ART PUEDE IR UN ART
        # Juan es el más listo
        if word[1].startswith('a') and anteriores[-1][1].startswith('r') and not (anteriores[-2][1].startswith('d') or anteriores[-2][1].startswith('n')):
            frase_con_mask.insert(-1,'[MASK]')
            #print(1)
            
        # SI ANTES DE UN ADJ O DE UN ADV NO HAY NI SUST NI ART PUEDE IR UN ART
        # los listos y los guapos // han ganado los rojos
        if (word[1].startswith('a') or anteriores[-1][1].startswith('r')) and not (anteriores[-1][1].startswith('d') or anteriores[-1][1].startswith('n')):
            frase_con_mask.insert(-1,'[MASK]')
            #print(2)
            
        # ART-PRONREL(QUE, CUAL, CUALES) = PRON REL COMPUESTO (EL QUE, EL CUAL, LOS CUALES)
        if word[0] in pronombres_relativos and not anteriores[-1][1].startswith('d'):
            frase_con_mask.append('[MASK]')
            #print(3)
        
        # ANTES DE UN SUST PUEDE IR UN ART
        if word[1].startswith('n'):
            frase_con_mask.append('[MASK]')
            #print(4)
        
        # Se appendea la palabra actual
        frase_con_mask.append(word[0])
        
        # Se appendea la palabra actual a la lista de anteriores
        anteriores.append(word)
        if print_word: print(word)
        
    frase_con_mask.append('[SEP]')
    return ' '.join(frase_con_mask)    

In [46]:
############### PREDICTOR ###############

def predictor(text):
    masked_indexes,index = [],1
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    index = 0
    for tup in zip(tokenized_text, indexed_tokens):
        if(tup[0] == '[MASK]'):
            masked_indexes.append(index)
        index+=1

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    predicted_tokens = []
    for masked_index in masked_indexes:   
        predicted_index = torch.argmax(predictions[0, masked_index]).item()
        predicted_tokens.append(tokenizer.convert_ids_to_tokens([predicted_index])[0])

    return predicted_tokens

# falta hacer que junte de/a+art y ponga del/al
def frase_final(text):
    predicted_tokens = predictor(text)
    c=0
    texto_final = []
    tokens_usados = []
    for word in text.split():
        if word == '[CLS]' or word == '[SEP]':
            continue
        if word == '[MASK]':
            if predicted_tokens[c] in articulos:
                texto_final.append(predicted_tokens[c])
                tokens_usados.append(predicted_tokens[c])
                c+=1
            else:
                c+=1
                continue
        else:
            texto_final.append(word)
        

    return ' '.join(texto_final),tokens_usados

In [52]:
############### OBTENCIÓN DE LA FRASE CORRECTA ###############

from nltk.tag import StanfordPOSTagger
import os

jar = '/home/raconma/TFM/stanford-postagger-full-2018-10-16/stanford-postagger-3.9.2.jar' 
modelo_tagger = '/home/raconma/TFM/stanford-postagger-full-2018-10-16/models/spanish.tagger'

pos_tagger = StanfordPOSTagger(modelo_tagger, jar, encoding='utf8')

frase = random.choice(lines)

frase_taggeada = pos_tagger.tag(frase.split())
print(frase_taggeada)
text = sentence_masker(frase_taggeada, False)
frase, tokens = frase_final(text)
print("Frase original: ", text)
print("Tokens: ", tokens)
print("Frase final: ", frase)

[('mejores', 'aq0000'), ('jugadores', 'nc0p000')]
Frase original:  [MASK] [CLS] mejores [MASK] jugadores [SEP]
Tokens:  []
Frase final:  mejores jugadores


In [55]:
############### OBTENCIÓN DE LA FRASE CORRECTA ###############

from nltk.tag import StanfordPOSTagger
import os

jar = '/home/raconma/TFM/stanford-postagger-full-2018-10-16/stanford-postagger-3.9.2.jar' 
modelo_tagger = '/home/raconma/TFM/stanford-postagger-full-2018-10-16/models/spanish.tagger'

pos_tagger = StanfordPOSTagger(modelo_tagger, jar, encoding='utf8')

frase = "Fui a Madrid de Austria"

frase_taggeada = pos_tagger.tag(frase.split())
print(frase_taggeada)
text = sentence_masker(frase_taggeada, False)
frase, tokens = frase_final(text)
print("Frase original: ", text)
print("Tokens: ", tokens)
print("Frase final: ", frase)



[('Fui', 'vsis000'), ('a', 'sp000'), ('Madrid', 'np00000'), ('de', 'sp000'), ('Austria', 'np00000')]
Frase original:  [CLS] Fui a [MASK] Madrid de [MASK] Austria [SEP]
Tokens:  []
Frase final:  Fui a Madrid de Austria
